**Question 2:**

For the data you selected for the first question, use Python to construct an ensemble classifier. You may choose any ensemble classifier.

*   Hold back the same 30% of test data as in the first question.
*   Select parameters for your ensemble based on the remaining data and attempt to build the best possible model.

(a) When your model is complete, compare the error rate on the 30% test data from the ensemble and the best decision tree classifier you obtained in the first question. Compute confusion matrices for the test data from each model and comments on any differences in performance.

(b) Which model do you prefer and why? Be brief and clear.

**Solution:**

Here, we can use the data used in first question to construct a random forest ensemble. To start with, we load the libraries and packages which will be used in the solution.


In [0]:
import pandas as pd
import numpy as np
# To import the models (Decision Tree Classifier and Regressor)
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier

# To display a tree
from sklearn.tree import plot_tree

# To measure accuracy
from sklearn import metrics, preprocessing, tree

from sklearn.model_selection import cross_validate, KFold

# To support plots
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from IPython.display import Image
from pylab import rcParams
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from io import StringIO
import warnings
from sklearn.metrics import accuracy_score
from scipy import misc
from matplotlib.pyplot import imread
from matplotlib.colors import ListedColormap
import math

# To display all the plots inline
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Binarizer
import seaborn as sn

Now, we can load the bank marketing data in the similar way. The data for the problem is loaded below:

In [0]:
# To load datasets
bank_data = pd.read_csv("./bank-additional-full.csv")
bank_data.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [0]:
bank_data["job"] = bank_data["job"].astype('category')
bank_data["marital"] = bank_data["marital"].astype('category')
bank_data["education"] = bank_data["education"].astype('category')
bank_data["default"] = bank_data["default"].astype('category')
bank_data["housing"] = bank_data["housing"].astype('category')
bank_data["loan"] = bank_data["loan"].astype('category')
bank_data["contact"] = bank_data["contact"].astype('category')
bank_data["month"] = bank_data["month"].astype('category')
bank_data["day_of_week"] = bank_data["day_of_week"].astype('category')
bank_data["poutcome"] = bank_data["poutcome"].astype('category')
bank_data["y"] = bank_data["y"].astype('category')

In [0]:
bank_data["job"] = bank_data["job"].cat.codes
bank_data["marital"] = bank_data["marital"].cat.codes
bank_data["education"] = bank_data["education"].cat.codes
bank_data["default"] = bank_data["default"].cat.codes
bank_data["housing"] = bank_data["housing"].cat.codes
bank_data["loan"] = bank_data["loan"].cat.codes
bank_data["contact"] = bank_data["contact"].cat.codes
bank_data["month"] = bank_data["month"].cat.codes
bank_data["day_of_week"] = bank_data["day_of_week"].cat.codes
bank_data["poutcome"] = bank_data["poutcome"].cat.codes
bank_data["y"] = bank_data["y"].cat.codes

bank_data.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,1,3,1,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,3,0,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,1,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,3,0,0,2,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


After the data is loaded, we can save the attributes and responses in separate variables to use to build the ensemble

In [0]:
x = bank_data.iloc[:,0:20]
y = bank_data.iloc[:,20:21]
y.shape

(41188, 1)

In [0]:
# To increase quality of figures
plt.rcParams["figure.figsize"] = (20, 10)

Now, we separate the 30% data for the testing to check the generaization error later on.

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 123)
y_train = y_train.values
x_train = x_train.values
x_train.shape

(28831, 20)

In [0]:
y_train.shape

(28831, 1)

After loading the data, we can build the random forest ensemble on specified training data. We can define multiple cases to check the behavior of our ensemble. 

**Case 1:**

In this case, we uses the best parameter's found from question 1 and check the accuracy of the enseble with those parameters. Refer the code and results below.

Also, we can also use the concept of calss imbalance here. As the data is imbalanced, we can use the class_weight argument to nullify the imbalance effect.

In [0]:
model_forest=RandomForestClassifier(class_weight= 'balanced',random_state=598, max_features='sqrt', max_depth=5, min_samples_split=3, n_estimators=1000)
model_forest.fit(x_train, y_train)

In [0]:
y_test_hat = model_forest.predict(x_test)

print('Accuracy:', metrics.accuracy_score(y_test, y_test_hat))

Accuracy: 0.9087966334870924


**Case 2:** 

Another possibility in case of ensemble is to use the gridsearch to find the best paramters from the specified range. To do so, we uses the following code. 

We define the range/possible_values for each argument such as n_estimaters, max_depth, and min_samples_split.


In [0]:
model_forest = GridSearchCV(RandomForestClassifier(class_weight= 'balanced',random_state=598, max_features='sqrt'),
                            
                            n_jobs=-1,
                            iid=False,
                            param_grid={
                                'n_estimators': [300, 400, 500],
                                'max_depth': [7, 8, 9,10,11,12],
                                'min_samples_split': [100, 150, 200]
                            })

In [0]:
model_forest.fit(x_train, y_train)
print('The parameters found by CV search:')
print(model_forest.best_params_)

C:\Users\skesarwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\skesarwa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


The parameters found by CV search:
{'max_depth': 12, 'min_samples_split': 100, 'n_estimators': 500}


In [0]:
y_test_hat = model_forest.predict(x_test)

print('Accuracy:', metrics.accuracy_score(y_test, y_test_hat))

Accuracy: 0.8602411588573279


**Case 3:**

Finally, we can make the full tree without specifying any hyperparameter value, since the random forest uses the randomized value and weight over the various trees, the overfitting does not happen very quickly. So, many times, it is advisable to use the trees to maximum limit. 

We can use the following code to check the accuracy in this case.


In [0]:
model_forest = RandomForestClassifier(class_weight= 'balanced',random_state=598, max_features='sqrt',n_estimators=1000)
model_forest.fit(x_train, y_train)

C:\Users\skesarwa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=1000, n_jobs=None, oob_score=False,
                       random_state=598, verbose=0, warm_start=False)

In [0]:
y_test_hat = model_forest.predict(x_test)

In [0]:
print('Accuracy:', metrics.accuracy_score(y_test, y_test_hat))

Accuracy: 0.9121145909201263


To Answer the part A of the problem, we can compare the accuracy of the ensemble in various defined cases. 

We observe that if we use the enseble with the trees to its maximum splitting limit, we get the better accuracy


Now, to choose for the model to answer the part b of the problem 2 . We choose the ensemble method with maximum split i.e. with no stopping criteria because while constructing ensemble, the overfitting averages out and the accuracy achieved is more than the case discussed in question 1 without overfitting